In [1]:
import json
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.palettes import Category20_20
output_notebook()
colors = Category20_20

Loading BokehJS ...

In [20]:
def plot_roc_run(idx, title):
    info = json.loads(open('runs/{}/info.json'.format(idx)).read())
    p = figure(title=title, plot_width=600, plot_height=400)
    metrics  = ['uncertainty_classifer', 'uncertainty_mean_argmax']
    c = 0
    for k in sorted(info['evaluations'][0].keys()):
        if not k.startswith('uncertainty') or k.endswith('auc'): continue
        if k not in metrics: continue
        xys = np.array([info['evaluations'][i][k] for i in range(len(info['evaluations']))])
        xs = np.mean(xys, axis=0)[:, 0]
        ys = np.mean(xys, axis=0)[:, 1]
        stds = np.std(xys, axis=0)[:, 1]

        # Bollinger shading glyph:
        auc = np.array([info['evaluations'][i][k+'_auc'] for i in range(len(info['evaluations']))])
        auc = np.mean(auc)
        band_x = np.append(xs, xs[::-1])
        band_y = np.append(ys - stds, (ys + stds)[::-1])
        p.patch(band_x, band_y, color=colors[c], fill_alpha=0.2)
        p.line(xs, ys, legend='{0}: {1:.2f}'.format(k, auc*100), color=colors[c], line_width=3)

        p.xaxis.axis_label = 'Dataset Proportion'
        p.yaxis.axis_label = 'Accuracy'
        #p.legend.location = 'top_right'
        p.legend.location = 'bottom_left'
        size = '11pt'
        p.xaxis.major_label_text_font = size
        p.xaxis.axis_label_text_font_size = size
        p.yaxis.major_label_text_font = size
        p.yaxis.axis_label_text_font_size = size
        
        c += 3

    show(p)

In [21]:
plot_roc_run(21, 'CIFAR100')

In [38]:
plot_roc_run(12, 'CIFAR10')

In [41]:
plot_roc_run(13, 'CIFAR100')